# DSO 530 Project

In [1]:
import math
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import time
import os

from glob import glob
from datetime import date, datetime, time, timedelta
from matplotlib import pyplot as plt
from pylab import rcParams
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from tqdm import tqdm_notebook

%matplotlib inline

#### Input params ##################
rawpath = './raw data 17-19/'
featurepath = './engineered features 17-19/'
cv_size = 0.3                   # proportion of dataset to be used as cross-validation set
Nmax = 15                       # for feature at day t, we use lags from t-1, t-2, ..., t-N as features
                                # Nmax is the maximum N we are going to test
fontsize = 14
ticklabelsize = 14
####################################

## Functions

In [2]:
def get_preds_lin_reg(df, target_col, N, pred_min, offset, forecast):
    """
    Given a dataframe, get prediction at timestep t using values from t-1, t-2, ..., t-N.
    Inputs
        df         : dataframe with the values you want to predict. Can be of any length.
        target_col : name of the column you want to predict e.g. 'close'
        N          : get prediction at timestep t using values from t-1, t-2, ..., t-N
        pred_min   : all predictions should be >= pred_min
        offset     : for df we only do predictions for df[offset:]. e.g. offset can be size of training set
    Outputs
        pred_list  : the predictions for target_col. np.array of length len(df)-offset.
    """
    # Create linear regression object
    regr = LinearRegression(fit_intercept=True)

    pred_list = []

    for i in range(offset, len(df['close'])):
        X_train = np.array(range(len(df['close'][i-N:i]))) # e.g. [0 1 2 3 4]
        y_train = np.array(df['close'][i-N:i]) # e.g. [2944 3088 3226 3335 3436]
        X_train = X_train.reshape(-1, 1)     # e.g X_train = 
                                             # [[0]
                                             #  [1]
                                             #  [2]
                                             #  [3]
                                             #  [4]]
        # X_train = np.c_[np.ones(N), X_train]              # add a column
        y_train = y_train.reshape(-1, 1)
    #     print X_train.shape
    #     print y_train.shape
    #     print 'X_train = \n' + str(X_train)
    #     print 'y_train = \n' + str(y_train)
        regr.fit(X_train, y_train)            # Train the model
        pred = regr.predict(np.array(N+(forecast-1)).reshape(1,-1))
    
        pred_list.append(pred[0][0])  # Predict the footfall using the model
    
    # If the values are < pred_min, set it to be pred_min
    pred_list = np.array(pred_list)
    pred_list[pred_list < pred_min] = pred_min
        
    return pred_list

def get_mape(y_true, y_pred): 
    """
    Compute mean absolute percentage error (MAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
# A function to get the best training period for each stock 
def get_best_N(mapeList, threshold = 0.03): 
    '''
    inputs
    mapeList : a list of mean absolute percentage errors, index 0 is where N = 2
    threshold : the threshold for how close the mape must be for N to be considered
    outputs:
    bestN : the maximum N that generates a mape as close as possible to the minimum
    '''
    min_mape = min(mapeList)
    arr = np.array(mapeList)
    arr = arr[arr < min_mape + threshold]
    best_N = mapeList.index(arr[-1]) + 2
    return(best_N)

In [3]:
# Predictor gets dataset and returns the trading dataset
def predictor(df, cv_size = 0.3, forecast = 2, start_day = 505):
    '''
    Splits data into training, testing, and validation sets, 
    then returns a test set with 
    Inputs:
    df: dataframe with time series stock prices
    cv_size: desired size of cross-validation set
    forecast: the number of days in advance to forecast
    '''
    # Split into train, cv, and test
    trade_start = df.index[df['day'] == start_day][0]
    cv_start = int((1-cv_size)*trade_start)
    test = df[trade_start:].copy()
    train = df[:cv_start].copy()
    cv = df[cv_start:trade_start].copy()
    train_cv = df[:trade_start].copy()
    # Saving Lengths of the dataframes for future use
    num_cv = len(cv)
    num_test = len(test)
    num_train = len(train)
    #Finding MAPE
    MAPE = []
    forecast = 2
    for N in  range(2, Nmax):
        est_list = get_preds_lin_reg(train_cv, 'close', N, 0, num_train, forecast)
        cv.loc[:, 'est_2day'] = est_list
        cv['pred'] = np.nan
        for i in range(cv_start+1, cv_start + len(cv)):
            cv.loc[i, 'pred'] = est_list[i-cv_start-1]
        cv.loc[cv_start,'pred'] = cv.loc[cv_start,'close']
        MAPE.append(get_mape(cv['close'], cv['pred']))
    N = get_best_N(MAPE)
    # Making Predictions for training set
    est_list = get_preds_lin_reg(train_cv, 'close', N, 0, num_train, forecast)
    cv.loc[:, 'est_2day'] = est_list
    cv['pred'] = np.nan
    for i in range(cv_start+1, cv_start + len(cv)):
        cv.loc[i, 'pred'] = est_list[i-cv_start-1]
    cv.loc[cv_start,'pred'] = cv.loc[cv_start,'close']
    # Getting Validation Error Metrics
    #r2_val = r2_score(cv['close'], cv['pred'])
    #rmse_val = math.sqrt(mean_squared_error(cv['close'], cv['pred'],))
    #mape_val = get_mape(cv['close'], cv['pred'])
    # Making Predictions for test set
    est_list = get_preds_lin_reg(df, 'close', N, 0, num_train + num_cv, forecast)
    test.loc[:, 'est_2day'] = est_list
    test['pred'] = np.nan
    for i in range(trade_start+1, trade_start + len(test)):
        test.loc[i, 'pred'] = est_list[i-trade_start-1]
    test.loc[trade_start,'pred'] = test.loc[trade_start,'close']
    #r2_test = r2_score(cv['close'], cv['pred'])
    #rmse_test = math.sqrt(mean_squared_error(cv['close'], cv['pred'],))
    #mape_test = get_mape(cv['close'], cv['pred'])
    return(test.reset_index(drop = True))

In [4]:
# defining function to load in data and create variables

def merger(rawpath, featurepath, file, training_period = 10, tech_start = 16):
    """
    Takes raw stock data and engineered features and
    returns a dataframe with new engineered variables
    """
    raw = pd.read_csv(rawpath + file).assign(filename=file)
    raw['day'] = raw.index + 1
    raw.columns = [str(x).lower().replace(' ', '_') for x in raw.columns]
    features = pd.read_csv(featurepath + file)
    featureCols = list(features.columns)
    featureCols[0] = 'time'
    features.columns = featureCols
    data = pd.merge(raw, features, on='time', how='outer')
    data['std_10PROClag'] = np.nan
    for i in range(tech_start + training_period, len(data)):
        std = data.loc[i-training_period:i-1, 'PROClag'].std()
        data.loc[i, 'std_10PROClag'] = std
    #data = data.iloc[16:, ].reset_index(drop = True)
    #cols = ['day','filename', 'PROClag', 'std_10PROClag', 'volume', 'close']
    return(data)


In [5]:
# Defining get metric function
def getMetric(df, day):
    '''
    gets the desired metric on a specified day
    '''
    proj_return = (df.loc[day, 'est_2day'] - df.loc[day, 'close'])/\
    df.loc[day, 'close']
    #metric = proj_return/(df.loc[day, 'std_10PROClag'])
    metric = proj_return
    return(metric)

In [6]:
def trade(day, cash, increment, threshold, trade_data, 
          holding, cost = 0, sell_thresh = -0.05):
    """
    Trades on a specific day given a starting amount of cash,
    A buying increment,
    A metric threshold,
    The day, and the dictionary of stock dataframes,
    And a dictionary of stocks held
    Inputs
    day: desired trading day
    cash: current cash level
    increment: how much cash to invest each day
    treshold: threshold for the chosen metric
    trade_data: dictionary of dataframes
    holding: dictionary of holdings for each stock
    """
    metricDF = pd.DataFrame(columns=['stock', 'metric'])
    for stock, df in trade_data.items():
        metric = getMetric(df, day)
        metricDF = metricDF.append({'stock': stock,
                                    'metric': metric}, ignore_index=True)
    metricDF = metricDF.sort_values(by='metric', ascending=False).reset_index(drop=True)
    # choosing Stocks to buy
    buyDF = metricDF.loc[metricDF['metric'] > threshold, ]
    for stock in buyDF['stock']:
        price = trade_data[stock].loc[day, 'close']
        shares = min(increment, cash)/price
        holding[stock] += shares
        cash -= shares*price*(1+cost)
    # Defining Stocks that meet Sell Condition
    sellDF = metricDF.loc[metricDF['metric'] <= sell_thresh, ]
    for stock in sellDF['stock']:
        price = trade_data[stock].loc[day, 'close']
        shares = holding[stock]
        cash += shares * price*(1-cost)
        holding[stock] = 0
    return(cash, holding)

## Loading Data

In [7]:
eng_files = sorted(os.listdir('./engineered features 17-19'))
raw_files = sorted(os.listdir('./raw data 17-19'))

In [8]:
# First Stock
# Combining Raw files and engineered features
doc = 'SH600000.csv'
raw = pd.read_csv(rawpath + doc)
raw['day'] = raw.index + 1
raw.columns = [str(x).lower().replace(' ', '_') for x in raw.columns]
features = pd.read_csv(featurepath + doc)
featureCols = list(features.columns)
featureCols[0] = 'time'
features.columns = featureCols
fullCombo = pd.merge(raw, features, on='time', how='outer')
# Change all column headings to be lower case, and remove spacing
#df.columns = [str(x).lower().replace(' ', '_') for x in df.columns]
#df['day'] = df.index + 1
#df.head(10)
#df = raw.copy()
#df.head()
#fullCombo.iloc[16:20,]
df = fullCombo.iloc[16: , ].reset_index(drop = True)

In [9]:
# stock_files. The list of stock files we have available
stock_files = sorted(glob(rawpath + '*.csv'))
for i in range(len(stock_files)):
    item = stock_files[i]
    item = item[item.rfind('/') + 1:]
    stock_files[i] = item
print(stock_files[:5])

['SH600000.csv', 'SH600010.csv', 'SH600015.csv', 'SH600016.csv', 'SH600018.csv']


In [10]:
# Combining All Datasets into one dictionary of many datasets
# Using merger function

stock_data = {}
for file in stock_files:
    data = merger(rawpath, featurepath, file)
    stock_data[file] = data
stock_data[stock_files[0]]

,time,open,high,low,close,volume,filename,day,volumelag,rsilag,fastKlag,fastDlag,ADlag,OBVlag,MA5lag,MA15lag,day5Returnlag,day15Returnlag,PROClag,std_10PROClag
0,Day1,154.6414,159.7405,154.4560,156.9592,45036400,SH600000.csv,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Day2,156.9592,158.7207,156.3102,156.5883,21043100,SH600000.csv,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Day3,156.5883,158.5353,155.8467,158.4426,23335200,SH600000.csv,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Day4,158.9988,162.1510,158.9988,159.5551,33835300,SH600000.csv,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Day5,159.5551,160.8530,158.6280,160.0186,29530100,SH600000.csv,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,Day752,157.8890,159.6702,157.5073,158.0162,40150148,SH600000.csv,752,23789908.0,62.404014,0.876540,0.864197,-0.500000,1.846001e+09,156.36230,152.918667,2.392746,4.198110,0.080595,0.945048
752,Day753,158.5252,158.5252,154.8356,155.2172,37033892,SH600000.csv,753,40150148.0,62.745694,0.847054,0.862597,-0.529428,1.886151e+09,157.12566,153.351233,2.390708,4.633532,0.080530,0.925471
753,Day754,155.3445,156.3623,155.2172,156.2351,21671028,SH600000.csv,754,37033892.0,51.626945,0.588230,0.770608,-0.793148,1.849117e+09,157.30376,153.631133,-1.533547,2.866762,-1.787213,0.930669
754,Day755,156.3623,156.3623,155.2172,155.7262,13678175,SH600000.csv,755,21671028.0,54.766169,0.658233,0.697839,0.777836,1.870788e+09,157.02386,153.987373,-0.967725,4.510676,0.653650,1.156130


## Testing Functions

In [11]:
#predictor(df).head()

In [12]:
#merger(rawpath, featurepath, doc).loc[10:15, ]

In [13]:
trade_data = {}
for file in stock_files:
    trade_data[file] = predictor(stock_data[file])
trade_data[stock_files[0]].loc[,['day', 'close', 'pred', 'est_2day']]

,time,open,high,low,close,volume,filename,day,volumelag,rsilag,...,ADlag,OBVlag,MA5lag,MA15lag,day5Returnlag,day15Returnlag,PROClag,std_10PROClag,est_2day,pred
0,Day505,132.5229,133.2632,130.6720,131.0422,24163832,SH600000.csv,505,19792340.0,51.756291,...,0.076866,1.523527e+09,133.18918,132.753193,-0.826427,2.857099,0.650225,1.142438,131.293393,131.042200
1,Day506,131.4123,133.0164,131.0422,133.0164,19746228,SH600000.csv,506,24163832.0,44.678998,...,-0.714264,1.499363e+09,132.52288,132.851907,-1.939036,1.142855,-1.680671,1.149565,130.601529,131.293393
2,Day507,131.7825,133.5100,130.0550,130.4252,19451676,SH600000.csv,507,19746228.0,51.082213,...,1.000000,1.519109e+09,132.39948,133.082233,0.559663,2.764487,1.495302,1.262117,131.381500,130.601529
3,Day508,130.6720,131.4123,130.0550,130.0550,13603633,SH600000.csv,508,19451676.0,43.899876,...,-0.785702,1.499658e+09,132.02930,133.148040,-1.491169,-0.094524,-1.967255,0.943816,130.597054,131.381500
4,Day509,129.9316,130.6720,124.7492,126.4767,30069914,SH600000.csv,509,13603633.0,42.970300,...,-1.000000,1.486054e+09,131.56040,133.115133,-2.407416,-0.846640,-0.284244,1.046512,129.843461,130.597054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,Day752,157.8890,159.6702,157.5073,158.0162,40150148,SH600000.csv,752,23789908.0,62.404014,...,-0.500000,1.846001e+09,156.36230,152.918667,2.392746,4.198110,0.080595,0.945048,160.906157,160.156443
248,Day753,158.5252,158.5252,154.8356,155.2172,37033892,SH600000.csv,753,40150148.0,62.745694,...,-0.529428,1.886151e+09,157.12566,153.351233,2.390708,4.633532,0.080530,0.925471,161.365032,160.906157
249,Day754,155.3445,156.3623,155.2172,156.2351,21671028,SH600000.csv,754,37033892.0,51.626945,...,-0.793148,1.849117e+09,157.30376,153.631133,-1.533547,2.866762,-1.787213,0.930669,158.343343,161.365032
250,Day755,156.3623,156.3623,155.2172,155.7262,13678175,SH600000.csv,755,21671028.0,54.766169,...,0.777836,1.870788e+09,157.02386,153.987373,-0.967725,4.510676,0.653650,1.156130,156.930229,158.343343


In [57]:
trade_data[stock_files[0]].loc[:,['day', 'close', 'pred', 'est_2day']]

,day,close,pred,est_2day
0,505,131.0422,131.042200,131.293393
1,506,133.0164,131.293393,130.601529
2,507,130.4252,130.601529,131.381500
3,508,130.0550,131.381500,130.597054
4,509,126.4767,130.597054,129.843461
...,...,...,...,...
247,752,158.0162,160.156443,160.906157
248,753,155.2172,160.906157,161.365032
249,754,156.2351,161.365032,158.343343
250,755,155.7262,158.343343,156.930229


## Final Results

In [36]:
# Testing Function over many days and returning cash
# Best Thresholds for now:
# threshold = 0.02
# sell_thresh = -0.05
cash = 1000000
increment = 20000
holding = {stock: 0 for stock in stock_files}
cost = 0.00065
total_assets = []
threshold = 0.02
sell_thresh = -0.05
#day = 0
#cash, holding = trade(day, cash, increment, threshold, trade_data, holding, cost)
#trade(day, cash, increment, threshold, trade_data, holding, cost)
#  Testing function over many days
totalDays = len(trade_data[stock_files[0]])
for day in range(0, totalDays-1):
    cash, holding = trade(day, cash, increment, threshold, 
                          trade_data, holding, cost, sell_thresh)
    assets = cash
    for stock, shares in holding.items():
        price = trade_data[stock].loc[day, 'close']
        assets += price*shares
    total_assets.append(assets)
    #print('day ', day, ':  ', cash)
# print(cash)
# print(holding)

for stock, shares in holding.items():
    price = trade_data[stock].loc[totalDays - 1, 'close']
    cash += price*shares* (1-cost)
    holding[stock] = 0
total_assets.append(cash)
print(f'Total cash: {cash:.2f}')
#print(total_assets)

Total cash: 1182041.70


In [37]:
# getting of days with profit and sharpe ratio
pnl_list = []
for i in range(1, len(total_assets)):
    pnl = (total_assets[i]/total_assets[i-1]) - 1
    pnl_list.append(pnl)
#print(pnl_list[:15])
pnl_arr = np.array(pnl_list)
sharpe = (np.sqrt(252) * pnl_arr.mean())/(pnl_arr.std())
profitable_days = np.sum(pnl_arr>0)
print(f'Sharpe Ratio:\t\t{sharpe:.4f}')
print(f'Total Profitable days:\t{profitable_days}')

Sharpe Ratio:		1.1013
Total Profitable days:	126


In [38]:
# Testing Function 
#cash = 1000000
#print(cash)
#holding = {stock: 0 for stock in stock_files}
#trade(0, cash, increment, threshold, trade_data, holding, cost)
#print(cash)
#print(holding)

In [39]:
# Getting Statistical Metrics for each dataframe
R2 = []
MAPE = []
RMSE = []
for df in trade_data.values():
    r2 = r2_score(df['close'], df['pred'])
    rmse = np.sqrt(mean_squared_error(df['close'], df['pred'],))
    mape = get_mape(df['close'], df['pred'])
    R2.append(r2); RMSE.append(rmse); MAPE.append(mape)
print(f'Mean Test R-Squared:\t{(sum(R2)/len(R2)):.4f}')
print(f'Mean Test RMSE:\t\t{(sum(RMSE)/len(RMSE)):.4f}')
print(f'Mean Test MAPE:\t\t{(sum(MAPE)/len(MAPE)):.4f}')

Mean Test R-Squared:	0.8253
Mean Test RMSE:		9.5396
Mean Test MAPE:		2.4773


## Further Exploration

In [18]:
# # A function to get the best training period for each stock 
# # Redefining to change the test set
# def get_best_N(mapeList, threshold = 0.03): 
#     '''
#     inputs
#     mapeList : a list of mean absolute percentage errors, index 0 is where N = 2
#     threshold : the threshold for how close the mape must be for N to be considered
#     outputs:
#     bestN : the maximum N that generates a mape as close as possible to the minimum
#     '''
#     min_mape = min(mapeList)
#     best_N = mapeList.index(min_mape) + 2
#     return(best_N)

In [19]:
# # Creating New Dictionary of dataframes
# trade_data2 = {}
# for file in stock_files:
#     trade_data2[file] = predictor(stock_data[file])
# trade_data2[stock_files[0]].head()

In [20]:
# # Getting Statistical Metrics for each dataframe
# # (New Predictions for Test)
# R2 = []
# MAPE = []
# RMSE = []
# for df in trade_data2.values():
#     r2 = r2_score(df['close'], df['pred'])
#     rmse = np.sqrt(mean_squared_error(df['close'], df['pred'],))
#     mape = get_mape(df['close'], df['pred'])
#     R2.append(r2); RMSE.append(rmse); MAPE.append(mape)
# print(f'Mean Test R-Squared:\t{(sum(R2)/len(R2)):.4f}')
# print(f'Mean Test RMSE:\t\t{(sum(RMSE)/len(RMSE)):.4f}')
# print(f'Mean Test MAPE:\t\t{(sum(MAPE)/len(MAPE)):.4f}')

In [21]:
# # Testing Function over many days and returning cash
# # Best Thresholds for now:
# # threshold = 0.02
# # sell_thresh = -0.05
# cash = 1000000
# increment = 20000
# holding = {stock: 0 for stock in stock_files}
# cost = 0.00065
# total_assets = []
# threshold = 0.02
# sell_thresh = -0.04
# #  Testing function over many days
# totalDays = len(trade_data2[stock_files[0]])
# for threshold in [0.02, 0.03, 0.04]:
#     for sell_thresh in [-0.05, -0.04, -0.03, 0.02, -0.01, 0, 0.01]:
#         cash = 1000000
#         print(cash)
#         for day in range(0, totalDays-1):
#             cash, holding = trade(day, cash, increment, threshold, 
#                                   trade_data2, holding, cost, sell_thresh)
#             assets = cash
#             for stock, shares in holding.items():
#                 price = trade_data2[stock].loc[day, 'close']
#                 assets += price*shares
#             total_assets.append(assets)
#         # Selling off at end
#         for stock, shares in holding.items():
#             price = trade_data2[stock].loc[totalDays - 1, 'close']
#             cash += price*shares* (1-cost)
#             holding[stock] = 0
#         total_assets.append(cash)
#         print(f'Buy: {threshold}\t Sell: {sell_thresh}')
#         print(f'Total cash: {cash:.2f}')

In [55]:
# Testing Function over many days and returning cash
# Testing wth new thresholds
# Best Thresholds for now:
# threshold = 0.02
# sell_thresh = -0.05
cash = 1000000
increment = 5000
holding = {stock: 0 for stock in stock_files}
#cost = 0.00065
cost = 0
total_assets = []
threshold = 0.04
sell_thresh = -0.02
#  Testing function over many days
totalDays = len(trade_data[stock_files[0]])
for day in range(0, totalDays-1):
    cash, holding = trade(day, cash, increment, threshold, 
                          trade_data, holding, cost, sell_thresh)
    assets = cash
    for stock, shares in holding.items():
        price = trade_data[stock].loc[day, 'close']
        assets += price*shares
    total_assets.append(assets)
# Selling off at end
for stock, shares in holding.items():
    price = trade_data[stock].loc[totalDays - 1, 'close']
    cash += price*shares* (1-cost)
    holding[stock] = 0
total_assets.append(cash)
print(f'Buy: {threshold}\t Sell: {sell_thresh}')
print(f'Total cash: {cash:.2f}')

Buy: 0.04	 Sell: -0.02
Total cash: 996463.68


In [49]:
# getting of days with profit and sharpe ratio
pnl_list = []
for i in range(1, len(total_assets)):
    pnl = (total_assets[i]/total_assets[i-1]) - 1
    pnl_list.append(pnl)
#print(pnl_list[:15])
pnl_arr = np.array(pnl_list)
sharpe = (np.sqrt(252) * pnl_arr.mean())/(pnl_arr.std())
profitable_days = np.sum(pnl_arr>0)
print(f'Sharpe Ratio:\t\t{sharpe:.4f}')
print(f'Total Profitable days:\t{profitable_days}')

Sharpe Ratio:		-0.4364
Total Profitable days:	94
